In [ ]:
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime, date, timedelta

In [ ]:
start = datetime(2019,1,1,0,0,0)
end = datetime(2023,2,20,23,59,59)

In [ ]:
def download_data(frequency, start_inner, end_inner):
    unix_start = str(int(time.mktime(time.strptime(f'{str(start_inner)}', '%Y-%m-%d %H:%M:%S'))) * 1000)
    unix_end = str(int(time.mktime(time.strptime(f'{str(end_inner)}', '%Y-%m-%d %H:%M:%S'))) * 1000)
    url = f"https://api4.binance.com/api/v3/klines?symbol=BTCUSDT&interval={frequency}&startTime={unix_start}&endTime={unix_end}&limit=1000"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    return response.json()

In [ ]:
data_list = []

start_inner = start
end_inner = start + timedelta(days=160)
data_list += download_data('4h', start_inner, end_inner)
while end_inner < end:
    start_inner = end_inner
    end_inner = start_inner + timedelta(days=160)
    data_list += download_data('4h', start_inner, end_inner)
else:
    start_inner = end_inner
    end_inner = end
    data_list += download_data('4h', start_inner, end_inner)

In [ ]:
value = np.array(data_list)
data = pd.DataFrame(value, columns=['time', 'open', 'high', 'low', 'close', 'volume', 'delete', 'delete', 'delete', 'delete', 'delete', 'delete'])

In [ ]:
data.drop('delete', axis=1, inplace=True)

In [ ]:
data['time'] = [datetime.fromtimestamp(int(i)/1000) for i in data['time'].values]

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.set_index('time', inplace=True)
for i in range(1, len(data.index)):
    if data.index[i] != data.index[i-1] + timedelta(hours=4):
        print(data.index[i-1], data.index[i])

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.to_csv('data/btcusdt_4h.csv', index=0)